# Um simples sistema de recomendação de filmes utilizando o algoritmo KNN

O Fluxo de Dados (Pipeline)
Entrada (Input): Usuário digita "Poderoso Chefão" (PT-BR).

Conversão (API): Seu código busca na API do TMDB por "Poderoso Chefão" e pede o título original ou o ID.

Retorno: The Godfather.

Processamento (KNN): O KNN procura vizinhos de The Godfather no seu dataset do Kaggle.

Retorno: Goodfellas, Scarface, Casino.

Saída (Output): Seu código pega essa lista em inglês e consulta a API novamente pedindo os títulos em pt-BR.

Exibição final: "Os Bons Companheiros", "Scarface", "Cassino".

# Pipeline:
* Passo 1: Executar as células de bibliotecas
* Passo 2: Limpa o dataset
* Passo 3: Treina o KNN
* Passo 4: Executa a classe de tradução
* Passo 5: Recebe o input do usuário
* Passo 6: Função de recomendação
* Passo 7: Ver as recomendações

In [ ]:
!pip install tmdbv3api

In [ ]:
import pandas as pd
import numpy as np
from tmdbv3api import TMDb, Movie
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Importação do dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ggtejas/tmdb-imdb-merged-movies-dataset")

print("Path to dataset files:", path)

In [ ]:
movie_dd = pd.read_csv("/root/.cache/kagglehub/datasets/ggtejas/tmdb-imdb-merged-movies-dataset/versions/300/TMDB  IMDB Movies Dataset.csv")

In [ ]:
pd.set_option('display.max_columns', None)
movie_dd.head()

In [ ]:
movie_dd.info()

#Filtragem do dataset



* Onehot encoding no gênero do filme
* Remover as colunas irrelevantes('revenue', 'budget', 'status', 'production_companies', 'production_countries', 'spoken_languages', 'overview')
* Remover os filmes com menos de 100 votos







In [ ]:
movie_dd.drop(['revenue', 'budget', 'status', 'production_companies', 'production_countries', 'spoken_languages', 'overview', 'homepage', 'backdrop_path', 'tagline', 'poster_path', 'tconst', 'adult'], axis=1, inplace=True, errors='ignore')
movie_dd.head()

In [ ]:
movie_dd.info()

# Filtrando por número de votos(>100) e fazendo o one hot encoding utilizando o dummies do pandas

In [ ]:
#Filtrando por filmes com mais de 100 votos
movie_dd_filtrado_votos = movie_dd[movie_dd['vote_count'] >= 100]
movie_dd_filtrado_votos = movie_dd_filtrado_votos.reset_index(drop=True)
#One hot encoding
movie_dd_filtrado_one_hot = movie_dd_filtrado_votos['genres'].str.get_dummies(sep=',')
movie_dd_filtrado_one_hot.head()

# Selecionando os valores numéricos e normalizando

In [ ]:
valores_numericos = movie_dd_filtrado_votos[['vote_average', 'popularity']]
scaler = StandardScaler()
valores_numericos_normalizados = scaler.fit_transform(valores_numericos)
numeric_df = pd.DataFrame(valores_numericos_normalizados, columns=['vote_average', 'popularity'])

dataset_para_knn = pd.concat([numeric_df, movie_dd_filtrado_one_hot], axis=1)

dataset_referencia = movie_dd_filtrado_votos[['original_title', 'title']]
dataset_para_knn.head()

# Treinando o KNN para a recomendação

In [ ]:
recomendacao_KNN = NearestNeighbors(n_neighbors=20,metric='cosine')
recomendacao_KNN.fit(dataset_para_knn)

NearestNeighbors(metric='cosine', n_neighbors=20)

# Tradução do input

In [ ]:
class Traducao:
  def __init__(self, api_key):
    #Configuração da API
    self.tmdb = TMDb()
    self.tmdb.api_key = api_key
    self.tmdb.language = 'pt-BR'
    self.movie_api = Movie()

  def busca_titulo_original(self, nome_filme_ptBr):
    try:
      filme_ptBr = self.movie_api.search(nome_filme_ptBr)
      if filme_ptBr:
        return filme_ptBr[0].original_title
      else:
        print(f"Filme não encontrado na API: {nome_filme_ptBr}")
        return None
    except Exception as e:
      print(f"Erro ao buscar título original: {e}")
      return None


  def busca_titulo_ptBr(self, titulo_original):
    try:
      filme_original = self.movie_api.search(titulo_original)
      if filme_original:
        return filme_original[0].title
      print(f"Filme não encontrado na API: {titulo_original}")
      return None
    except Exception as e:
      print(f"Erro ao buscar título original: {e}")
      return None


  def processar_lista_avaliacoes(self, lista_filmes_usuario):
    dados_para_KNN = {}
    for item in lista_filmes_usuario:
      nome_ptBr = item['filme']
      nota = item['nota_filme']
      titulo_original = self.busca_titulo_original(nome_ptBr)

      if titulo_original:
        dados_para_KNN[titulo_original] = nota
    return dados_para_KNN

# Input do usuário

In [ ]:
minha_api_key = 'Aqui entra a chave de API da TMDB'
tradutor = Traducao(minha_api_key)

filmes_avaliados = []
qnt_filmes = input("Quantos filmes deseja avaliar?\n")


for i in range(int(qnt_filmes)):
  filme = input(f"Qual o {i+1}º filme deseja avaliar?\n")
  nota_filme = float(input(f"Qual a nota de {filme} (0-10)?\n"))
  # Uma lista de dicionário - Por quê python é doido?
  filmes_avaliados.append({
      'filme': filme,
      'nota_filme': nota_filme
      })

for filme_info in filmes_avaliados:
  print(f"Filme: {filme_info['filme']}, Nota: {filme_info['nota_filme']}")


dados_para_o_knn = tradutor.processar_lista_avaliacoes(filmes_avaliados)
print(dados_para_o_knn)

Quantos filmes deseja avaliar?
7
Qual o 1º filme deseja avaliar?
Senhor dos anéis as duas torres
Qual a nota de Senhor dos anéis as duas torres (0-10)?
10
Qual o 2º filme deseja avaliar?
Senhor dos anéis o retorno do rei
Qual a nota de Senhor dos anéis o retorno do rei (0-10)?
10
Qual o 3º filme deseja avaliar?
Batman o cavaleiro das trevas
Qual a nota de Batman o cavaleiro das trevas (0-10)?
10
Qual o 4º filme deseja avaliar?
Top gun maverick
Qual a nota de Top gun maverick (0-10)?
9
Qual o 5º filme deseja avaliar?
F1 o filme
Qual a nota de F1 o filme (0-10)?
9
Qual o 6º filme deseja avaliar?
Jurassic Park O Parque dos Dinossauros
Qual a nota de Jurassic Park O Parque dos Dinossauros (0-10)?
8
Qual o 7º filme deseja avaliar?
Tarzan
Qual a nota de Tarzan (0-10)?
10
Filme: Senhor dos anéis as duas torres, Nota: 10.0
Filme: Senhor dos anéis o retorno do rei, Nota: 10.0
Filme: Batman o cavaleiro das trevas, Nota: 10.0
Filme: Top gun maverick, Nota: 9.0
Filme: F1 o filme, Nota: 9.0
Filme: 

# Função para a recomendação

In [ ]:
def recomendar_filmes(dados_usuario, df_ref, df_numerico, modelo):
    """
    dados_usuario: Dicionário dos inputs do usuário
    df_ref: dataset_referencia (Só Textos)
    df_numerico: dataset_para_knn
    modelo: recomendacao_KNN o modelo treinado
    """

    perfil_usuario = np.zeros(df_numerico.shape[1])
    total_peso = 0
    encontrados = 0

    print("\n--- Calculando Perfil do Usuário ---")

    for titulo, nota in dados_usuario.items():
        # Busca o filme no dataset de referência ojgando tudo para minúsculo para evitar erro
        match = df_ref[df_ref['original_title'].str.lower() == titulo.lower()]

        if len(match) > 0:
            idx = match.index[0]

            # Pega os dados numéricos desse filme no OUTRO dataset
            vetor_filme = df_numerico.iloc[idx].values

            # A Matemática da coisa: Soma o vetor do filme multiplicado pela nota (Peso)
            peso_real = nota - 5.0
            perfil_usuario += vetor_filme * peso_real
            total_peso += abs(peso_real)
            encontrados += 1
            print(f"Filme incluído: {titulo} com a nota do usuário. (Nota: {nota})")
        else:
            print(f"Filme não existe no Dataset local: {titulo}")

    if encontrados == 0:
        return []

    # 2. Normalizar o vetor do usuário (Média Ponderada)
    perfil_usuario = perfil_usuario / total_peso

    df_perfil_usuario = pd.DataFrame([perfil_usuario], columns=df_numerico.columns)

    # Vizinhos do filme usando o KNN
    distancias, indices = modelo.kneighbors(df_perfil_usuario)

    # 4. Traduzir os índices numéricos de volta para nomes de filmes
    recomendacoes = []
    for i in indices[0]:
        # Pega o título na tabela de referência usando o índice que o KNN devolveu
        titulo = df_ref.iloc[i]['original_title']

        # Filtro: Não recomendar o que o usuário já viu
        if titulo not in dados_usuario:
            recomendacoes.append(titulo)

    return recomendacoes

In [ ]:
print("Traduzindo inputs...")

dados_input_knn = tradutor.processar_lista_avaliacoes(filmes_avaliados)

# Gerando a Recomendação ---
if len(dados_input_knn) > 0:
    sugestoes = recomendar_filmes(
        dados_input_knn,
        dataset_referencia,   # Tabela de nomes
        dataset_para_knn,     # Tabela de números (Z-Score + OneHot)
        recomendacao_KNN      # Seu modelo treinado
    )

    print("\n" + "="*40)
    print("🎬 FILMES RECOMENDADOS PARA VOCÊ 🎬")
    print("="*40)

    for i, filme_original in enumerate(sugestoes[:10]):
        # Traduz de volta para Português só para exibir bonito
        nome_pt = tradutor.busca_titulo_ptBr(filme_original)
        print(f"{i+1}. {nome_pt} ({filme_original})")

else:
    print("Nenhum filme válido foi encontrado para gerar recomendações.")

Traduzindo inputs...

--- Calculando Perfil do Usuário ---
Filme incluído: The Lord of the Rings: The Two Towers com a nota do usuário. (Nota: 10.0)
Filme incluído: The Lord of the Rings: The Return of the King com a nota do usuário. (Nota: 10.0)
Filme incluído: The Dark Knight com a nota do usuário. (Nota: 10.0)
Filme incluído: Top Gun: Maverick com a nota do usuário. (Nota: 9.0)
Filme não existe no Dataset local: F1
Filme incluído: Jurassic Park com a nota do usuário. (Nota: 8.0)
Filme incluído: Tarzan com a nota do usuário. (Nota: 10.0)

🎬 FILMES RECOMENDADOS PARA VOCÊ 🎬
1. O Senhor dos Anéis: A Sociedade do Anel (The Lord of the Rings: The Fellowship of the Ring)
2. Harry Potter e o Prisioneiro de Azkaban (Harry Potter and the Prisoner of Azkaban)
3. Vingadores: Guerra Infinita (Avengers: Infinity War)
4. Vingadores: Ultimato (Avengers: Endgame)
5. Interestelar (Interstellar)
6. Guerra nas Estrelas (Star Wars)
7. Piratas do Caribe: A Maldição do Pérola Negra (Pirates of the Caribbe